<a href="https://colab.research.google.com/github/alexandreblima/IC-THS/blob/master/mnist_0_sgd_epoch200_batch128_2layers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# arquivo: mnist_0_sgd_epoch200_batch128_2layers.ipynb   
# Adaptação dos códigos do livro "Deep Learning with TensorFlow2
# and Keras: Regression, Convnets, GANs RNNs, NLP and more with
# TensorFlow2 and the Keras API, de Antonio Gulli, Amita Kapoor 
# e Sujit Pal, 2a ed., Packt. 
# otimizador = algoritmo do gradiente estocástico ou
# Stochastic Gradient Descent (SGD)
# código para TensorFlow 2

# Uso do TensorFlow 2 para definir uma rede que reconhece dígitos 
# manuscritos MNIST (http://yann.lecun.com/exdb/mnist/)
# O MNIST é um banco de dados de dígitos manuscritos composto de um conjunto 
# de treinamento de 60.000 exemplos (amostras) e um conjunto de testes de 10.000 exemplos. 
# Os exemplos de treinamento são anotados por humanos com a resposta correta. 
# Por exemplo, se o dígito manuscrito for o número "3", "3" será o rótulo (label) 
# associado a esse exemplo. Cada imagem MNIST está codificada em escala de cinza e 
# consiste em uma matriz de 28 x 28 pixels, em que cada pixel é um número inteiro 
# na faixa [0, 255]

# Seguindo o estilo Keras, o TensorFlow 2 fornece bibliotecas adequadas 
# (https://www.tensorflow.org/api_docs/python/tf/keras/datasets) para carregar o 
# conjunto de dados e dividi-lo em conjuntos de treinamento, X_train, usado para 
# ajustar a rede, e conjuntos de testes, X_test, usados para avaliar o desempenho. 
# Os dados são convertidos para o tipo "float32" para usar precisão de 32 bits 
# ao treinar uma rede neural e normalizados para o intervalo [0, 1]. 
# Além disso, os rótulos de treino e de teste são carregados em Y_train e Y_test, 
# respectivamente. Na sequência, os rótulos são vetorizados para o formato de 
# codificação "one-hot". 
# Exemplo de codificação one-hot do dígito 3 (feature categórica ou não-numérica):
# v = [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]

import tensorflow as tf
#import numpy as np
from tensorflow import keras
print("Tensorflow version " + tf.__version__)
import datetime

# comente as linhas abaixo se usar o programa no Spyder 4.0
%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

# Parâmetros da rede e de treinamento
epocas = 200   # define a duração do treinamento
lote_tam = 128 # número de amostras que alimentarão a rede em uma dada época
               # de treinamento (em inglês, lote = batch)
verbose = 1
n_classes = 10 # número de saídas = número de dígitos
val_split = 0.2 # a fração do número de amostras de treinamento reservadas para
                # validação => 48.000 amostras para treino + 12.000 amostras para 
                # validação = 60.000 exemplos do conjunto de treinamento MNIST     

# carrega a base de dados MNIST
mnist = keras.datasets.mnist
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Em Python, uma lista é uma sequência de objetos separados por vírgulas. 
# Os objetos podem ser de qualquer tipo: números, strings e até mesmo outras listas. 
# Por exemplo, digite a linha seguinte no Console:
# animais = ['peixe', 'gato', 'cão']
# A variável "animais" é uma lista de objetos de string
# Indexação dos itens da lista "animais":
# Digite as seguintes linhas no Console
# animais[0]
# animais[1]
# animais[2]
# animais[-3]
# animais[-2]
# animais[-1]

# X_train, Y_train, X_test e Y_test são listas numéricas designadas por "array". 
# X_train possui 60.000 imagens de dígitos manuscritos. Cada imagem está em escala 
# de cinza e consiste em uma matriz de 28 x 28 pixels, em que cada pixel é um número 
# inteiro na faixa [0, 255] (representação com 8 bits). O mesmo se aplica a X_test.

# X_train e X_test serão reformatados para 60.000 x 784 (28 x 28 = 784)
# Ou seja, X_train e X_test serão "vetorizados" para o formato de vetor x de tamanho 784 × 1;

reformat = 784 # é a dimensão do espaço de "features"
               # ou seja, a camada de entrada possui 784 "input units"
               # Uma entrada é um vetor coluna 784 x 1
 
X_train = X_train.reshape(60000, reformat) 
X_test = X_test.reshape(10000, reformat) # "reshape" é um método de "numpy.ndarray"
                                         # que retorna um "array" que contém os mesmos
                                         # dados, só que em um outro formato

# A representação dos números contidos nos "arrays" X_train e X_test será convertida
# para codificação em ponto flutuante com 32 bits. Para maiores detalhes, consulte:
# http://www.ic.uff.br/~simone/scminter/contaulas/6_FLOAT.pdf 

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalização em [0,1]
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'amostras de treinamento')
print(X_test.shape[0], 'amostras de teste')

# Representação "one-hot" dos rótulos (labels) de treinamento e teste
Y_train = tf.keras.utils.to_categorical(Y_train, n_classes)
Y_test = tf.keras.utils.to_categorical(Y_test, n_classes)

# construção do modelo
# A camada de saída é composta por 10 neurônios com função de ativação "softmax", que é uma 
# generalização da função sigmóide ou logística. 

model = tf.keras.models.Sequential()
model.add(keras.layers.Dense(n_classes, input_shape=(reformat,), kernel_initializer='zeros', name='dense_layer', activation='softmax'))

# sumário do modelo
model.summary()

# Depois de definir o modelo, é preciso compilá-lo para que ele possa ser executado pelo TensorFlow 2

# Opções a serem feitas durante a compilação. 

# 1) Selecionar um otimizador, que é o algoritmo específico usado para atualizar os pesos da rede enquanto o modelo é treinado. 
# vide: https://www.tensorflow.org/api_docs/python/tf/keras/optimizers

# 2) Selecionar uma função objetivo, que é usada pelo otimizador para navegar no espaço de pesos. 
# Nota: frequentemente, a função objetivo é designada por função de perda ("loss function") ou função custo, 
# sendo o processo de otimização definido como um processo de minimização dessa função. 
# vide: https://www.tensorflow.org/api_docs/python/tf/keras/losses

# 3) Avaliar o modelo treinado.

# compilando o modelo
model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Tensorboard
# comente as linhas abaixo se usar o programa no Spyder 4.0
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, write_graph=True)

# treinamento do modelo no TensorFlow 2
treino = model.fit(X_train, Y_train, batch_size=lote_tam, epochs=epocas, verbose=verbose, validation_split=val_split, callbacks=tensorboard_callback)
# model.fit retorna o objeto "treino", que é do tipo "History".  

treino_dic = treino.history 
# "treino.history" retorna um contêiner de dicionário (uma generalização do conceito de lista): tipo "dict" 
# Um dicionário contém pares de (chave, valor)
treino_dic.keys()
# chaves no tf 2.1.0: ['loss', 'accuracy', 'val_loss', 'val_accuracy']

import matplotlib.pyplot as plt

font = {'family': 'serif',
        'color':  'darkred',
        'weight': 'normal',
        'size': 16,
        }

acc = treino.history['accuracy']
val_acc = treino.history['val_accuracy']
loss = treino.history['loss']
val_loss = treino.history['val_loss']

epochs = range(len(acc))

plt.clf()

plt.plot(epochs, acc, 'b', label='Acurácia de treinamento')
plt.plot(epochs, val_acc, 'r', label='Acurácia de validação')
plt.title('Curvas de aprendizado', fontdict=font)
plt.xlabel('tempo (épocas)', fontdict=font)
plt.ylabel('Acurácia', fontdict=font)
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'b', label='Erro de treinamento')
plt.plot(epochs, val_loss, 'r', label='Erro de validação')
plt.title('Curvas de aprendizado ', fontdict=font)
plt.xlabel('tempo (épocas)', fontdict=font)
plt.ylabel('Erro', fontdict=font)
plt.legend()

plt.show()

# avaliação do modelo
test_loss, test_acc = model.evaluate(X_test, Y_test)
print('Acurácia do teste:', test_acc)

# previsões
predictions = model.predict(X_test)

# comente as linhas abaixo se usar o programa no Spyder 4.0
%tensorboard --logdir logs/fit